## Prep

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [3]:
import pandas as pd

# Erstellen des DataFrames
F_supermarket = pd.DataFrame({
    'Milch': [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
    'Käse': [1, 1, 0, 0, 0, 0, 1, 0, 1, 1],
    'Butter': [1, 1, 1, 1, 1, 0, 1, 0, 1, 0],
    'Brot': [1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
    'Kaffee': [0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
    'Zucker': [1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
    'Mehl': [1, 1, 1, 0, 0, 1, 1, 0, 0, 1]
})

# Ausgabe des DataFrames
print(F_supermarket)


   Milch  Käse  Butter  Brot  Kaffee  Zucker  Mehl
0      1     1       1     1       0       1     1
1      0     1       1     1       1       1     1
2      1     0       1     0       1       1     1
3      1     0       1     0       0       0     0
4      1     0       1     0       1       0     0
5      1     0       0     0       0       0     1
6      1     1       1     1       1       1     1
7      0     0       0     0       0       0     0
8      1     1       1     0       0       1     0
9      1     1       0     1       1       1     1


#### SupportF

In [22]:
def intersection_all_elements(lst):
    result = (F_supermarket[lst[0]] == 1)
    for i in range(1, len(lst)):
        result &=  (F_supermarket[lst[i]] == 1)
    return result

def supportF_regel(list1, list2):
    return intersection_all_elements(list1+list2).mean()

def supportF(list1):
    return supportF_regel(list1, [])

a = supportF(['Zucker', 'Mehl'])
b = supportF_regel(['Zucker', 'Mehl'], ['Butter'])

In [23]:
a

0.5

In [24]:
b

0.4

#### ConF

In [28]:
def conF(list1, list2):
    return supportF(list1+list2)/supportF(list1)

conF(['Brot'], ['Käse'])

1.0

## Algorithmus

### Teil 1

In [29]:
minsupp = 0.4
minconf = 0.9

In [6]:
def get_frequent_items(lst_of_combinations):
    df = pd.DataFrame(columns = ['item', 'frequency'])
    i = 0

    if(isinstance(lst_of_combinations[0], list)):
        for e in lst_of_combinations:
            temp = supportF(e)
            if temp >= 0.4:
                df.loc[i,'item'] = e
                df.loc[i,'frequency'] = temp
                i+=1
    else:
        for e in lst_of_combinations:
            temp = supportF([e])
            if temp >= 0.4:
                df.loc[i,'item'] = e
                df.loc[i,'frequency'] = temp
                i+=1
    
    
    return df

def combinations(lst, n):
    if n>2:
        # Flatten the list
        flat_list = [item for sublist in lst for item in sublist]
        
        # Get all unique items
        unique_items = list(set(flat_list))

        list_of_tuples = list(itertools.combinations(unique_items, n))
        # Generate all combinations of 3 items
        return [list(tup) for tup in list_of_tuples] 
    else: 
        combinations = []
        for i in range(len(lst)):
            for j in range(i+1, len(lst)):
            # Append each 2-element combination to the list
                combinations.append([lst[i], lst[j]])
        return combinations


In [7]:
df_1_item = get_frequent_items(F_supermarket.columns)

In [8]:
combis = combinations(df_1_item['item'].tolist(), 2)

df_2_item = get_frequent_items(combis)
df_2_item

,item,frequency
0,"[Milch, Käse]",0.4
1,"[Milch, Butter]",0.6
2,"[Milch, Kaffee]",0.4
3,"[Milch, Zucker]",0.5
4,"[Milch, Mehl]",0.5
5,"[Käse, Butter]",0.4
6,"[Käse, Brot]",0.4
7,"[Käse, Zucker]",0.5
8,"[Käse, Mehl]",0.4
9,"[Butter, Kaffee]",0.4


In [9]:
combis = combinations(df_2_item['item'].tolist(), 3)
df_3_item = get_frequent_items(combis)
df_3_item

,item,frequency
0,"[Käse, Brot, Zucker]",0.4
1,"[Käse, Brot, Mehl]",0.4
2,"[Käse, Zucker, Butter]",0.4
3,"[Käse, Zucker, Mehl]",0.4
4,"[Käse, Zucker, Milch]",0.4
5,"[Brot, Zucker, Mehl]",0.4
6,"[Zucker, Butter, Mehl]",0.4
7,"[Zucker, Butter, Milch]",0.4
8,"[Zucker, Kaffee, Mehl]",0.4
9,"[Zucker, Mehl, Milch]",0.4


In [10]:
combis = combinations(df_2_item['item'].tolist(), 4)
df_4_item = get_frequent_items(combis)
df_4_item

,item,frequency
0,"[Käse, Brot, Zucker, Mehl]",0.4


#### Power set:

In [12]:
def power_set(input_set):
    # Base case: an empty set has one subset, the empty set
    if len(input_set) == 0:
        return [[]]

    # Recursive case:
    # - take an element from the set
    # - find all subsets of the set without this element
    # - the power set is the set of subsets without this element plus the set of subsets with this element
    subsets = power_set(input_set[1:])
    return subsets + [[input_set[0]] + subset for subset in subsets]

def power_set_reduced(my_set):
    lst = power_set(my_set)
    del lst[0]
    del lst[-1]
    return lst


X = df_4_item['item'][0]
powerset = power_set_reduced(X)
powerset.sort(key=len)

print("X: ",X)
print("powerset: ",powerset)

X:  ['Käse', 'Brot', 'Zucker', 'Mehl']
powerset:  [['Mehl'], ['Zucker'], ['Brot'], ['Käse'], ['Zucker', 'Mehl'], ['Brot', 'Mehl'], ['Brot', 'Zucker'], ['Käse', 'Mehl'], ['Käse', 'Zucker'], ['Käse', 'Brot'], ['Brot', 'Zucker', 'Mehl'], ['Käse', 'Zucker', 'Mehl'], ['Käse', 'Brot', 'Mehl'], ['Käse', 'Brot', 'Zucker']]


### Teil 2

In [30]:
# Convert X to a set for faster operations
X_set = set(X)

# Generate association rules
association_rules = []
for p in powerset:
    temp = conF(X, p)
    if temp >= minconf:
        association_rules.append((list(X_set - set(p)), p))

# Print the association rules
for rule in association_rules:
    print(f"{rule[0]} --> {rule[1]} = {conF(rule[0], rule[1])}")
    print()


['Zucker', 'Käse', 'Brot'] --> ['Mehl'] = 1.0

['Käse', 'Mehl', 'Brot'] --> ['Zucker'] = 1.0

['Zucker', 'Käse', 'Mehl'] --> ['Brot'] = 1.0

['Zucker', 'Mehl', 'Brot'] --> ['Käse'] = 1.0

['Käse', 'Brot'] --> ['Zucker', 'Mehl'] = 1.0

['Zucker', 'Käse'] --> ['Brot', 'Mehl'] = 0.8

['Käse', 'Mehl'] --> ['Brot', 'Zucker'] = 1.0

['Zucker', 'Brot'] --> ['Käse', 'Mehl'] = 1.0

['Mehl', 'Brot'] --> ['Käse', 'Zucker'] = 1.0

['Zucker', 'Mehl'] --> ['Käse', 'Brot'] = 0.8

['Käse'] --> ['Brot', 'Zucker', 'Mehl'] = 0.8

['Brot'] --> ['Käse', 'Zucker', 'Mehl'] = 1.0

['Zucker'] --> ['Käse', 'Brot', 'Mehl'] = 0.6666666666666667

['Mehl'] --> ['Käse', 'Brot', 'Zucker'] = 0.6666666666666667

